In [1]:
import numpy as np
from bs4 import BeautifulSoup, Comment
import requests
from urllib.parse import urljoin, urlsplit
from queue import Queue
import re
from url_normalize import url_normalize
import urllib3
from glob import glob
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_similarity
from IPython.core.display import display, HTML
import networkx as nx
import matplotlib.pyplot as plt

# 1. Crawling

In [2]:
# Disable InsecureRequestWarning
urllib3.disable_warnings()

In [3]:
# Simple URL normalization
# Converts relative URLs to absolute and converts to lowercase, then applies url-normalize
# See (https://pypi.org/project/url-normalize/)

def norm_url(url, base):
    if url[0] == '/':
        return url_normalize(urljoin(base, url).lower())
    return url_normalize(url)

In [4]:
# Retrieve the base of a URL

def get_base_url(url):
    split = urlsplit(url)
    base = split.scheme + '://' + split.netloc + '/'
    return base

In [5]:
# Checks for outlinks in a HTML page

def find_outlinks(soup):
    # URL of the file is saved as the first comment of the HTML file.
    base = get_base_url(soup.findAll(text = lambda text: isinstance(text, Comment))[0])
    
    all_urls = [link.get('href') for link in soup.find_all('a', href=True)]
    
    # Selects only URLs that are either absolute or relative to the domain.
    selected_urls = [u for u in all_urls if len(u) > 1 and (u[0] == '/' and u[1] != '/' or u[0] == 'h')]
    
    # Normalizes selected URLs.
    all_urls_normalized = [norm_url(u, base) for u in selected_urls]
    
    return all_urls_normalized

In [6]:
# Finds robots.txt and processes its rules

def process_robots(url):
    base = get_base_url(url)
    robots_file = base + 'robots.txt'
    req = requests.get(robots_file)
    
    # If robots.txt cannot be processes, no rules are applied
    if req.status_code != 200:
        return []
    
    robots = req.text.split('\n')
    
    # Only check for 'User-agent: *'
    if 'User-agent: *' not in robots:
        return []
    
    robots = robots[robots.index('User-agent: *')+1:]

    # Retrieve all Disallow rules
    rules = []
    for rule in robots:
        if rule.startswith('Disallow: '):
            rules.append(rule.split(' ')[1])
        if rule.startswith('User-agent: '):
            break

    # Normalize all URLs in the rules
    rules_normalized = [norm_url(rule, base) for rule in rules]
    
    return rules_normalized

In [7]:
# Check for visible text only

def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    elif element == '\n' or element == ' ':
        return False
    return True

def extract_shingles(soup, n=4):
    # Extract all textual data
    data = soup.findAll(text=True)
    
    # Filter out comments, scripts, etc.
    texts = filter(visible, data)
    # Remove whitespaces
    texts = [str(e).strip() for e in texts]
    
    # Convert to list of words
    words = [text.split() for text in texts]
    words = [w for sub in words for w in sub]
    
    shingles = []
    for i in range(len(words) - n + 1):
        shingles.append(tuple(words[i:i+n]))
        
    return set(shingles)

In [8]:
def near_duplicate(shingles1, shingles2, threshold=0.8):
    union = len(set(list(shingles1) + list(shingles2)))
    overlap = len(list(shingles1) + list(shingles2)) - union

    return overlap/union > threshold

In [9]:
i = 1
n_pages = 100

q = Queue()
seed_url = "https://en.wikipedia.org/wiki/Alexandria_Ocasio-Cortez"

q.put(seed_url)

# Save texts of processed pages for near-duplication detection
processed = []

while i <= n_pages:
    # Get URL from queue
    url = q.get()

    # Make a request to URL
    try:
        req = requests.get(url, verify=False, timeout=5)
    except Exception:
        continue
    
    # Check if page exists and is accessible
    if req.status_code != 200:
        continue
    
    soup = BeautifulSoup(req.text)
    
    # Extract 4-shingles out of textual data
    shingles = extract_shingles(soup, n=4)
    
    # Check for near duplicates among processed files
    if any([near_duplicate(shingles, processed_file, threshold=0.8) for processed_file in processed]):
        continue
    
    # Save shingles for near-duplicate detection
    processed.append(shingles)
    
    # Insert the URL as a comment on the first line of the created file
    soup.insert(0, '\n')
    soup.insert(0, Comment(url))
    
    # Save the HTML of retrieved file
    with open('Pages/{}'.format('{}.html'.format(i)), 'wb+') as file:
        file.write(soup.prettify('utf-8'))
                
    # Find all outlinks in the file
    urls = find_outlinks(soup)
    # Retrieve rules from robots.txt
    rules = process_robots(url)
    
    # Use only outlinks that aren't contradictory to rules from robots.txt
    urls_to_add = []
    for u in urls:
        for rule in rules:
            if rule.startswith(u):
                continue
        urls_to_add.append(u)
    
    # Put filtered outlinks to queue
    for u in urls_to_add:
        q.put(u)
    
    i += 1

Corners cut:
- Some outlinks are discarded for easier processing
- Crawler cares only about 'Disallow' rules in robots.txt for any user-agent
- Near-duplicate detection is done using Jaccard similarity without using super-shingles
- URLs are added into queue without reprioritizing, Mercator's scheme isn't implemented

# 2. Indexing

In [10]:
# Check for visible text only

def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    elif element == '\n' or element == ' ':
        return False
    return True

def tokenize_text(soup):
    # Extract all textual data
    data = soup.findAll(text=True)
    
    # Filter out comments, scripts, etc.
    texts = filter(visible, data)
    # Remove whitespaces
    texts = [word_tokenize(str(e).strip()) for e in texts]
    
    # Convert to list of words
    words = [w.lower() for sub in texts for w in sub]
    
    # Remove words of length one
    words = [w for w in words if len(w) > 1]
    
    return words

In [11]:
# Get all files crawled in previous step
files = glob('Pages/*')

docterm = {}
words = []

ps = PorterStemmer()

for file in files:
    with open(file, 'r') as f:
        # Scan file content
        soup = BeautifulSoup(f)

        # Tokenize text
        tokenized = tokenize_text(soup)

        # Prepare english stoplist
        stopwords_list = stopwords.words('english')

        # Filter all words according to stoplist
        filtered = [word for word in tokenized if word not in stopwords_list]

        # Stem the rest of the words
        stemmed = [ps.stem(word) for word in filtered]

        # Save the list of stemmed words of the file
        docterm[file.split('/')[-1].split('.')[0]] = stemmed
        
        # Keep all the seen words
        words = words + stemmed

words = set(words)

In [12]:
# Create the final term-document matrix

termdoc = {}

for word in words:
    termdoc[word] = []
    
    for id, dictionary in docterm.items():
        if word in dictionary:
            termdoc[word].append(id)

In [13]:
# Output the term-document matrix to a file

output_file = 'BinaryInvertedIndex'

with open(output_file, 'w+') as f:
    for term, documents in termdoc.items():
        f.write('{}: {}\n'.format(term, documents))

Corners cut:
- Tokenizing doesn't check for language, encoding and assumes HTML format
- No normalization is performed
- Synonyms and homonyms aren't explicitly taken care of
- Inverted index is implemented without positional indexes, i.e. doesn't explicitly handle bi-grams

# 3. Ranking (content-based)

## 3.1 Creating multidimensional vector space

In [14]:
# Check for visible text only

def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    elif element == '\n' or element == ' ':
        return False
    return True

def tokenize_text(soup):
    # Extract all textual data
    data = soup.findAll(text=True)
    
    # Filter out comments, scripts, etc.
    texts = filter(visible, data)
    # Remove whitespaces
    texts = [word_tokenize(str(e).strip()) for e in texts]
    
    # Convert to list of words
    words = [w.lower() for sub in texts for w in sub]
    
    # Remove words of length one
    words = [w for w in words if len(w) > 1]
    
    return words

In [15]:
# Get all files crawled in previous step
files = glob('Pages/*')

docterm = {}
words = []

ps = PorterStemmer()

for file in files:
    with open(file, 'r') as f:
        # Scan file content
        soup = BeautifulSoup(f)

        # Tokenize text
        tokenized = tokenize_text(soup)

        # Prepare english stoplist
        stopwords_list = stopwords.words('english')

        # Filter all words according to stoplist
        filtered = [word for word in tokenized if word not in stopwords_list]

        # Stem the rest of the words
        stemmed = [ps.stem(word) for word in filtered]

        # Save the list of stemmed words of the file
        docterm[file.split('/')[-1].split('.')[0]] = stemmed
        
        # Keep all the seen words
        words = words + stemmed

words = set(words)

In [16]:
# Create the final term-document matrix

termdoc = {}

for word in words:
    termdoc[word] = {}
    
    for id, dictionary in docterm.items():
        if word in dictionary:
            # Assign ID of the document with the count of the word
            termdoc[word][id] = dictionary.count(word)

In [17]:
# Number of documents
N = len(files)
# Document frequency of terms
df_t = {k: len(v) for k, v in termdoc.items()}

# Inverted document frequency of terms
idf_t = {k: np.log10(N/v) for k, v in df_t.items()}

# Recalculate term frequency
tf_td = {}

for term in termdoc.keys():
    tf_td[term] = {k: (1+np.log10(v)) for k, v in termdoc[term].items()}

# Calculate tf-idf
tfidf = {}

for term in tf_td.keys():
    tfidf[term] = {k: (v*idf_t[term]) for k, v in tf_td[term].items()}

## 3.2 Representing queries

In [18]:
def process_query(q):
    # Tokenize
    tokens = word_tokenize(q)
    
    # Apply stoplist
    stopwords_list = stopwords.words('english')
    tokens_stopped = [t for t in tokens if t not in stopwords_list]
    
    # Stem
    ps = PorterStemmer()
    stems = [ps.stem(t) for t in tokens_stopped]
    
    return stems

In [19]:
query = 'Alexandria Ocasio-Cortez'

# Tokenize, stem and remove stopwords from query
processed = process_query(query)

# Term count
term_count = {k: (1+np.log10(processed.count(k))) for k in processed}
# Term weight basedd on idf_t
term_weight = {k: ((v*idf_t[k]) if k in idf_t else 0) for k, v in term_count.items()}

In [20]:
# Create vectors for computing cosine similarity

N = len(files)
docterm_tfidf = {}

for i in range(1, N+1):
    docterm_tfidf[str(i)] = {k: (tfidf[k][str(i)] if str(i) in tfidf[k] else 0) for k in words}

query_tfidf = {k: (term_weight[k] if k in term_weight else 0) for k in words}

## 3.3 Finding top-$k$ results

In [21]:
# Compute cosine similarity between all documents and searched query

similarities = cosine_similarity(
    [list(v.values()) for v in docterm_tfidf.values()], 
    np.array(list(query_tfidf.values())).reshape(1, -1)
)

# Index similarities with documents IDs
similarities_indexed = {str(i): similarities[i-1][0] for i in range(1, N+1)}

In [22]:
# Number of results to return
k = 10

# All results sorted
results = OrderedDict(sorted(similarities_indexed.items(), key=lambda x: x[1], reverse=True))

# Top-k results sorted
top_k_results = list(results.items())[:k]

In [23]:
# Show results

display(HTML('<b>Best result:</b>'))
print('Pages/{}.html'.format(top_k_results[0][0]))
display(HTML('<b>Other good results:</b>'))
for page in top_k_results[1:]:
    print('Pages/{}.html'.format(page[0]))

Pages/11.html


Pages/40.html
Pages/73.html
Pages/20.html
Pages/16.html
Pages/1.html
Pages/86.html
Pages/60.html
Pages/9.html
Pages/85.html


## 3.4 Pruning

(by considering only docs containing atleast one query term)

In [24]:
N = len(files)
docterm_tfidf_p = {}

# Skips documents that don't contain any query term
for word in processed:
    for i in range(1, N+1):
        if str(i) in tfidf[word]:
            docterm_tfidf_p[str(i)] = {k: (tfidf[k][str(i)] if str(i) in tfidf[k] else 0) for k in words}

query_tfidf = {k: (term_weight[k] if k in term_weight else 0) for k in words}

In [25]:
similarities_p = cosine_similarity(
    [list(v.values()) for v in docterm_tfidf_p.values()], 
    np.array(list(query_tfidf.values())).reshape(1, -1)
)

# Index similarities with documents IDs
similarities_indexed_p = {list(docterm_tfidf_p.keys())[i]: similarities_p[i][0] for i in range(0, len(similarities_p))}

In [26]:
# Number of results to return
k = 10

# All results sorted
results = OrderedDict(sorted(similarities_indexed.items(), key=lambda x: x[1], reverse=True))

# Top-k results sorted
top_k_results = list(results.items())[:k]

In [27]:
# Results are the same as without pruning

display(HTML('<b>Best result:</b>'))
print('Pages/{}.html'.format(top_k_results[0][0]))
display(HTML('<b>Other good results:</b>'))
for page in top_k_results[1:]:
    print('Pages/{}.html'.format(page[0]))

Pages/11.html


Pages/40.html
Pages/73.html
Pages/20.html
Pages/16.html
Pages/1.html
Pages/86.html
Pages/60.html
Pages/9.html
Pages/85.html


# 4. Ranking (link-based)

## 4.1 PageRank

### 4.1.1 Calculating PageRank

In [28]:
files = glob('Pages/*')

In [29]:
def remove_anchors_and_protocol(link):
    if '#' in link:
        link = link[:link.index('#')]
    if '://' in link:
        link = link[link.find('://') + len('://'):]
    return link

In [30]:
# Save URLs of files to build a graph

urls = {}

for file in files:
    with open(file, 'r') as f:
        soup = BeautifulSoup(f)
        urls[remove_anchors_and_protocol(soup.findAll(text = lambda text: isinstance(text, Comment))[0])] = file

In [31]:
# Create directed graph

g = nx.DiGraph()
g.add_nodes_from(files)

In [32]:
# Fill graph with edges

for file in files:
    with open(file, 'r') as f:
        soup = BeautifulSoup(f)
        outlinks = find_outlinks(soup)
        no_anchors = [remove_anchors_and_protocol(link) for link in outlinks]

        for link in no_anchors:
            if link in urls:
                g.add_edge(file, urls[link])

In [33]:
# Apply PageRank on the graph
pr = nx.pagerank(g, alpha=0.85)

In [34]:
# Sort by PageRank and show top-k

k = 10

pr_sorted = OrderedDict(sorted(pr.items(), key=lambda x: x[1], reverse=True))
top_k_pr = list(pr_sorted.items())[:k]

display(HTML('<b>PageRank:</b>'))
for page, pr_val in top_k_pr:
    print('{}: {}'.format(page, pr_val))

Pages/2.html: 0.19729884849841883
Pages/55.html: 0.043012226278560106
Pages/10.html: 0.04177481739955622
Pages/8.html: 0.025086382088624876
Pages/67.html: 0.024866114825023117
Pages/57.html: 0.023543162196014884
Pages/53.html: 0.023171097779716536
Pages/54.html: 0.022713330231858737
Pages/56.html: 0.02215577088102767
Pages/5.html: 0.020557085033451036


### 4.1.2 Finding top-$k$ results considering PageRank

In [35]:
query = 'Alexandria Ocasio-Cortez'

# Tokenize, stem and remove stopwords from query
processed = process_query(query)

# Term count
term_count = {k: (1+np.log10(processed.count(k))) for k in processed}
# Term weight basedd on idf_t
term_weight = {k: ((v*idf_t[k]) if k in idf_t else 0) for k, v in term_count.items()}

In [37]:
N = len(files)
docterm_tfidf_p = {}

# Skips documents that don't contain any query term
for word in processed:
    for i in range(1, N+1):
        if str(i) in tfidf[word]:
            docterm_tfidf_p[str(i)] = {k: (tfidf[k][str(i)] if str(i) in tfidf[k] else 0) for k in words}

query_tfidf = {k: (term_weight[k] if k in term_weight else 0) for k in words}

In [38]:
similarities_p = cosine_similarity(
    [list(v.values()) for v in docterm_tfidf_p.values()], 
    np.array(list(query_tfidf.values())).reshape(1, -1)
)

# Index similarities with documents IDs
similarities_indexed_p = {list(docterm_tfidf_p.keys())[i]: similarities_p[i][0] for i in range(0, len(similarities_p))}

In [39]:
# Normalize similarities to have same weight as PageRank

sim_max = max(list(similarities_indexed_p.values()))
sim_min = min(list(similarities_indexed_p.values()))

sim_norm = {'Pages/{}.html'.format(k): (v-sim_min)/(sim_max-sim_min) for k, v in similarities_indexed_p.items()}

In [40]:
# Prune and normalize PageRanks

pr_pruned = {k: v for k, v in pr.items() if k in sim_norm.keys()}

pr_max = max(list(pr_pruned.values()))
pr_min = min(list(pr_pruned.values()))

pr_norm = {k: (v-pr_min)/(pr_max-pr_min) for k, v in pr_pruned.items()}

In [41]:
# PageRank + Similarity
sim_pr = {k: (v+pr_norm[k])/2 for k, v in sim_norm.items()}

In [42]:
# Top-k results

k = 10

sim_pr_sorted = OrderedDict(sorted(sim_pr.items(), key=lambda x: x[1], reverse=True))
top_k_sim_pr = list(sim_pr_sorted.items())[:k]

display(HTML('<b>Best result:</b>'))
print(top_k_sim_pr[0][0])
display(HTML('<b>Other good results:</b>'))
for page in top_k_sim_pr[1:]:
    print(page[0])

Pages/40.html


Pages/8.html
Pages/11.html
Pages/73.html
Pages/33.html
Pages/85.html
Pages/16.html
Pages/20.html
Pages/13.html
Pages/78.html


## 4.2 HITS

In [43]:
query = 'Alexandria Ocasio-Cortez'

# Tokenize, stem and remove stopwords from query
processed = process_query(query)

# Term count
term_count = {k: (1+np.log10(processed.count(k))) for k in processed}
# Term weight basedd on idf_t
term_weight = {k: ((v*idf_t[k]) if k in idf_t else 0) for k, v in term_count.items()}

In [44]:
N = len(files)
docterm_tfidf_p = {}

# Skips documents that don't contain any query term
for word in processed:
    for i in range(1, N+1):
        if str(i) in tfidf[word]:
            docterm_tfidf_p[str(i)] = {k: (tfidf[k][str(i)] if str(i) in tfidf[k] else 0) for k in words}

query_tfidf = {k: (term_weight[k] if k in term_weight else 0) for k in words}

In [45]:
similarities_p = cosine_similarity(
    [list(v.values()) for v in docterm_tfidf_p.values()], 
    np.array(list(query_tfidf.values())).reshape(1, -1)
)

# Index similarities with documents IDs
similarities_indexed_p = {'Pages/{}.html'.format(list(docterm_tfidf_p.keys())[i]): similarities_p[i][0] for i in range(0, len(similarities_p))}

In [46]:
# Number of results to return
t = 15

# All results sorted
results = OrderedDict(sorted(similarities_indexed_p.items(), key=lambda x: x[1], reverse=True))

# Root sset
top_t_results = list(results.items())[:t]

In [47]:
# Only URLs that are in the root set
urls_p = {k: v for k, v in urls.items() if v in [res[0] for res in top_t_results]}

In [48]:
g = nx.DiGraph()
g.add_nodes_from([res[0] for res in top_t_results])

In [49]:
# Add neighbours to create base set

for file in [res[0] for res in top_t_results]:
    with open(file, 'r') as f:
        soup = BeautifulSoup(f)
        outlinks = find_outlinks(soup)
        no_anchors = [remove_anchors_and_protocol(link) for link in outlinks]

        for link in no_anchors:
            if link in urls:
                urls_p[link] = urls[link]

In [50]:
for k, v in urls.items():
    with open(v, 'r') as f:
        soup = BeautifulSoup(f)
        outlinks = find_outlinks(soup)
        no_anchors = [remove_anchors_and_protocol(link) for link in outlinks]

        for link in no_anchors:
            if link in urls_p:
                g.add_edge(v, urls_p[link])

In [51]:
# Apply HITS on the graph
h, a = nx.hits(g)

In [52]:
# Sort and print top-k authorities

k = 10

a_sorted = OrderedDict(sorted(a.items(), key=lambda x: x[1], reverse=True))
top_k_a = list(a_sorted.items())[:k]
    
display(HTML('<b>Authorities:</b>'))
for page, a_val in top_k_a:
    print('{}: {}'.format(page, a_val))

Pages/55.html: 0.0477804784019622
Pages/67.html: 0.035448733294485286
Pages/56.html: 0.032744503507629176
Pages/53.html: 0.03208615203694629
Pages/2.html: 0.03167806686756724
Pages/54.html: 0.031030548337235425
Pages/5.html: 0.027779397744287778
Pages/71.html: 0.02771408634762611
Pages/8.html: 0.025749759710674114
Pages/85.html: 0.021922514550108643


In [53]:
# Sort and print top-k hubs

k = 10

h_sorted = OrderedDict(sorted(h.items(), key=lambda x: x[1], reverse=True))
top_k_h = list(h_sorted.items())[:k]

display(HTML('<b>Hubs:</b>'))
for page, h_val in top_k_h:
    print('{}: {}'.format(page, h_val))

Pages/1.html: 0.0687927726271976
Pages/59.html: 0.03606287184992874
Pages/67.html: 0.028116959793936167
Pages/55.html: 0.026924234485432286
Pages/48.html: 0.02515987441609795
Pages/76.html: 0.024528906876039726
Pages/61.html: 0.024236884618082682
Pages/54.html: 0.024022396512391277
Pages/71.html: 0.02377848620053737
Pages/12.html: 0.022498951121742914
